In [15]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import col
from pyspark.sql.functions import mean, min, max, avg
from pyspark.sql.functions import col
from pyspark.sql import functions as F  

sc.install_pypi_package("pandas==0.25.1") #Install pandas version 0.25.1 
sc.install_pypi_package("matplotlib", "https://pypi.org/simple") #Install matplotlib from given PyPI repository

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Package already installed for current Spark context!
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/context.py", line 1110, in install_pypi_package
    raise ValueError("Package already installed for current Spark context!")
ValueError: Package already installed for current Spark context!



In [16]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer,VectorAssembler, VectorIndexer 
from pyspark.ml.feature import  StringIndexer, OneHotEncoder
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, CrossValidatorModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import col, mean, min, max
from pyspark.sql import functions as F  
from pyspark.sql import Window

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Load ML libraries

In [4]:

from pyspark.ml.classification import DecisionTreeClassifier as   DT      # decision tree
from pyspark.ml.classification import RandomForestClassifier, RandomForestClassificationModel              # Random Forest, ofcourse
from pyspark.ml.classification import MultilayerPerceptronClassifier      # feedforward artificial neural network
from pyspark.ml.classification import GBTClassifier                       # gradient-boosted tree 
from pyspark.ml.classification import                          # naive bayes
from pyspark.ml.classification import MultilayerPerceptronClassificationModel 
from pyspark.ml.classification import LogisticRegression                 # logistic regression

from pyspark.ml.regression import LinearRegression                       # Regression problem
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.regression import RandomForestRegressor 
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.regression import GeneralizedLinearRegression

from pyspark.ml.feature import PCA                                       # Principal Components Analysis 
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.classification import LogisticRegressionModel

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Use Spark command to load the Marketing data, from S3 bucket into your cluster, via the Jupyter Notebook.


In [11]:
df = spark.read.format("csv").options(header='true', inferSchema='true').load('s3://aws-emr-resources-257184607963-us-east-2/Marketing.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'campaign', 'y']

In [18]:
df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+------------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+---+
|age|         job|marital|education|default|balance|housing|loan|contact|day|month|campaign|  y|
+---+------------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+---+
| 58|  management|married| tertiary|     no|   2143|    yes|  no|unknown|  5|  may|       1| no|
| 44|  technician| single|secondary|     no|     29|    yes|  no|unknown|  5|  may|       1| no|
| 33|entrepreneur|married|secondary|     no|      2|    yes| yes|unknown|  5|  may|       1| no|
| 47| blue-collar|married|  unknown|     no|   1506|    yes|  no|unknown|  5|  may|       1| no|
| 33|     unknown| single|  unknown|     no|      1|     no|  no|unknown|  5|  may|       1| no|
+---+------------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+---+
only showing top 5 rows

In this data y is a variable that indicates whether a client is subscribed to a term deposit. Given that the goal here is to build a predictive model to predict, for new clients, their likelihood of subscribing to term deposit...
Identify the type of Machine learning style in this use case.

A: This will be a classification problem, predicting yes/no of subscribing to a term deposit.

##Prepare the data in readiness to use with Spark ML libraries

In [19]:
df.dtypes

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('age', 'int'), ('job', 'string'), ('marital', 'string'), ('education', 'string'), ('default', 'string'), ('balance', 'int'), ('housing', 'string'), ('loan', 'string'), ('contact', 'string'), ('day', 'int'), ('month', 'string'), ('campaign', 'int'), ('y', 'string')]

A. Convert all the string fields to numeric ones by using the StringIndexer transformer.

In [20]:
stringIndexer_label = StringIndexer(inputCol="y", outputCol="Label")

strIndexer_job = StringIndexer(inputCol="job", outputCol="job_IX")
strIndexer_marital = StringIndexer(inputCol="marital", outputCol="marital_IX")
strIndexer_edu = StringIndexer(inputCol="education", outputCol="education_IX")
strIndexer_default = StringIndexer(inputCol="default", outputCol="default_IX")
strIndexer_housing = StringIndexer(inputCol="housing", outputCol="housing_IX")
strIndexer_loan = StringIndexer(inputCol="loan", outputCol="loan_IX")
strIndexer_contact = StringIndexer(inputCol="contact", outputCol="contact_IX")
strIndexer_month = StringIndexer(inputCol="month", outputCol="month_IX")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

B. One Hot encoding

In [21]:

OneHot_job = OneHotEncoder(inputCol="job_IX", outputCol="job_Vec")
OneHot_marital = OneHotEncoder(inputCol="marital_IX", outputCol="marital_Vec")
OneHot_edu = OneHotEncoder(inputCol="education_IX", outputCol="education_Vec")
OneHot_default = OneHotEncoder(inputCol="default_IX", outputCol="default_Vec")
OneHot_housing = OneHotEncoder(inputCol="housing_IX", outputCol="housing_Vec")
OneHot_loan = OneHotEncoder(inputCol="loan_IX", outputCol="loan_Vec")
OneHot_contact = OneHotEncoder(inputCol="contact_IX", outputCol="contact_Vec")
OneHot_month = OneHotEncoder(inputCol="month_IX", outputCol="month_Vec")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
C. Create a feature vector by combining all features together.

In [22]:
VA_features = VectorAssembler(inputCols=["job_Vec", "marital_Vec", "education_Vec", 
                                         "default_Vec", "housing_Vec", "loan_Vec",
                                         "contact_Vec", "month_Vec", 
                                         'age', 'balance', 'day', 'campaign',], outputCol="Features")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
Build a Pipeline

In [23]:
pipe = Pipeline(stages=[stringIndexer_label, strIndexer_job, strIndexer_marital, strIndexer_edu, strIndexer_default, 
                        strIndexer_housing, strIndexer_loan, strIndexer_contact, strIndexer_month, 
                        OneHot_job, OneHot_marital, OneHot_edu, OneHot_default, OneHot_housing, OneHot_loan, OneHot_contact,
                        OneHot_month, 
                        VA_features]
               )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
pipedModel = pipe.fit(df)
dfTransformed = pipedModel.transform(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
dfTransformed.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'campaign', 'y', 'Label', 'job_IX', 'marital_IX', 'education_IX', 'default_IX', 'housing_IX', 'loan_IX', 'contact_IX', 'month_IX', 'job_Vec', 'marital_Vec', 'education_Vec', 'default_Vec', 'housing_Vec', 'loan_Vec', 'contact_Vec', 'month_Vec', 'Features']

In [26]:
#In your notebook display the first 5 records showing the vectorized features and label columns
dfTransformed.select(['Features', 'Label']).show(5, truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------------------------------------------------------------------------+-----+
|Features                                                                                       |Label|
+-----------------------------------------------------------------------------------------------+-----+
|(36,[1,11,14,16,17,18,20,21,32,33,34,35],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,58.0,2143.0,5.0,1.0])|0.0  |
|(36,[2,12,13,16,17,18,20,21,32,33,34,35],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,44.0,29.0,5.0,1.0])  |0.0  |
|(36,[7,11,13,16,17,20,21,32,33,34,35],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,33.0,2.0,5.0,1.0])          |0.0  |
|(36,[0,11,16,17,18,20,21,32,33,34,35],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,47.0,1506.0,5.0,1.0])       |0.0  |
|(36,[12,16,18,20,21,32,33,34,35],[1.0,1.0,1.0,1.0,1.0,33.0,1.0,5.0,1.0])                       |0.0  |
+-----------------------------------------------------------------------------------------------+-----+
only showing top 5 rows

In [27]:
df.select(['age', 'job', 'marital']).show(5, truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+------------+-------+
|age|job         |marital|
+---+------------+-------+
|58 |management  |married|
|44 |technician  |single |
|33 |entrepreneur|married|
|47 |blue-collar |married|
|33 |unknown     |single |
+---+------------+-------+
only showing top 5 rows

In [ ]:
### Split data into training and testing

In [28]:
print('\nSplit dataset into training and testing sets...')
training, testing = dfTransformed.select(["Label", "Features"]).randomSplit([0.7, 0.3])

### Split data, randomly, into training and test sets. Stratified sampling highly recommended,for skewed data

# training, testing = data.randomSplit([0.7, 0.3])
# #print len(training)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


Split dataset into training and testing sets...

In [ ]:
## Model 1: Random Forest

Define the model

In [29]:
RF = RandomForestClassifier(labelCol="Label", featuresCol="Features")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Fit/train the model

In [30]:
modelRF = RF.fit(training)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
## Make predictions
predictions = modelRF.transform(testing)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
predictions.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Label', 'Features', 'rawPrediction', 'probability', 'prediction']

Model evaluation

In [34]:
preds = predictions.select(['Label', 'prediction'])
print ('='*100)
print ('Confusion Matrix::')
print ('\n')
preds.crosstab('label', 'prediction').show(truncate = False)
print ('='*100)
print ('\n')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Confusion Matrix::


+----------------+-----+
|label_prediction|0.0  |
+----------------+-----+
|1.0             |1668 |
|0.0             |11994|
+----------------+-----+


Other metrics...

In [35]:
# f1|precision|recall|weightedPrecision|weightedRecall
evaluatorRF = MulticlassClassificationEvaluator(labelCol="Label", predictionCol="prediction", metricName="weightedPrecision")
accuracy = evaluatorRF.evaluate(predictions)

evaluator = MulticlassClassificationEvaluator(labelCol='Label', predictionCol='prediction', metricName='f1')  
F1_score = evaluator.evaluate(predictions)

evaluator = MulticlassClassificationEvaluator(labelCol='Label', predictionCol='prediction', metricName='weightedRecall')  
recall = evaluator.evaluate(predictions)

print("Accuracy = %g" % accuracy)
print("Test Error = %g" % (1.0 - accuracy))
print('The F1 Score = %g' % F1_score)
print('Recall = %g' % recall)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy = 0.770725
Test Error = 0.229275
The F1 Score = 0.820833
Recall = 0.87791

## Model 2: Logistic Regression


In [36]:
## Define the model
lr = LogisticRegression(labelCol="Label", featuresCol="Features")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
## fit the model
lrModel = lr.fit(training)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
## Make predictions
predLR = lrModel.transform(testing)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
preds = predLR.select(['label', 'prediction'])
print ('='*100)
print ('Confusion Matrix::')
print ('\n')
preds.crosstab('label', 'prediction').show(truncate = False)
print ('='*100)
print ('\n')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Confusion Matrix::


+----------------+-----+---+
|label_prediction|0.0  |1.0|
+----------------+-----+---+
|1.0             |1493 |175|
|0.0             |11816|178|
+----------------+-----+---+


In [50]:
evaluatorLR = MulticlassClassificationEvaluator(labelCol="Label", predictionCol="prediction", metricName="weightedPrecision")
accuracy = evaluatorRF.evaluate(predLR)

evaluator = MulticlassClassificationEvaluator(labelCol='Label', predictionCol='prediction', metricName='f1')  
F1_score = evaluator.evaluate(predLR)

evaluator = MulticlassClassificationEvaluator(labelCol='Label', predictionCol='prediction', metricName='weightedRecall')  
recall = evaluator.evaluate(predLR)

print("Accuracy = %g" % accuracy)
print("Test Error = %g" % (1.0 - accuracy))
print('The F1 Score = %g' % F1_score)
print('Recall = %g' % recall)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy = 0.839952
Test Error = 0.160048
The F1 Score = 0.841077
Recall = 0.87769

In [ ]:
## Model #3 Decision Tree Classifier

In [51]:
## Define the model
dtc = DT(labelCol="Label", featuresCol="Features")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
## fit the model
dtcModel = dtc.fit(training)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
## Make predictions
preddtc = dtcModel.transform(testing)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
preds = preddtc.select(['label', 'prediction'])
print ('='*100)
print ('Confusion Matrix::')
print ('\n')
preds.crosstab('label', 'prediction').show(truncate = False)
print ('='*100)
print ('\n')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Confusion Matrix::


+----------------+-----+---+
|label_prediction|0.0  |1.0|
+----------------+-----+---+
|1.0             |1606 |62 |
|0.0             |11949|45 |
+----------------+-----+---+


In [56]:
evaluatorDT = MulticlassClassificationEvaluator(labelCol="Label", predictionCol="prediction", metricName="weightedPrecision")
accuracy = evaluatorDT.evaluate(preddtc)

evaluator = MulticlassClassificationEvaluator(labelCol='Label', predictionCol='prediction', metricName='f1')  
F1_score = evaluator.evaluate(preddtc)

evaluator = MulticlassClassificationEvaluator(labelCol='Label', predictionCol='prediction', metricName='weightedRecall')  
recall = evaluator.evaluate(preddtc)

print("Accuracy = %g" % accuracy)
print("Test Error = %g" % (1.0 - accuracy))
print('The F1 Score = %g' % F1_score)
print('Recall = %g' % recall)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy = 0.844639
Test Error = 0.155361
The F1 Score = 0.829707
Recall = 0.879154

## Model #4 GBT Classifier  

In [61]:
## Define the model
GBT = GBTClassifier(labelCol="Label", featuresCol="Features")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [62]:
## fit the model
GBTModel = GBT.fit(training)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
## Make predictions
predgbt = GBTModel.transform(testing)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [64]:
preds = predgbt.select(['label', 'prediction'])
print ('='*100)
print ('Confusion Matrix::')
print ('\n')
preds.crosstab('label', 'prediction').show(truncate = False)
print ('='*100)
print ('\n')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Confusion Matrix::


+----------------+-----+---+
|label_prediction|0.0  |1.0|
+----------------+-----+---+
|1.0             |1530 |138|
|0.0             |11887|107|
+----------------+-----+---+


In [65]:
evaluatorgbt = MulticlassClassificationEvaluator(labelCol="Label", predictionCol="prediction", metricName="weightedPrecision")
accuracy = evaluatorDT.evaluate(predgbt)

evaluator = MulticlassClassificationEvaluator(labelCol='Label', predictionCol='prediction', metricName='f1')  
F1_score = evaluator.evaluate(predgbt)

evaluator = MulticlassClassificationEvaluator(labelCol='Label', predictionCol='prediction', metricName='weightedRecall')  
recall = evaluator.evaluate(predgbt)

print("Accuracy = %g" % accuracy)
print("Test Error = %g" % (1.0 - accuracy))
print('The F1 Score = %g' % F1_score)
print('Recall = %g' % recall)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy = 0.846567
Test Error = 0.153433
The F1 Score = 0.838969
Recall = 0.880179

## Logistic Regression seems to be the best, with the highest F1 score of 0.841

Since we have a high number of actual negatives in our data, I think F1 score is the best measure of success in this situation because it helps control for the uneven class distribution.